In [1]:
from scipy.stats import hypergeom
import statsmodels.stats.multitest as multi
import pandas as pd
import Bio
from Bio import Seq, SeqIO

In [2]:
finfafile = '../../../../figshare/annotation/genes/Elysia_crispata_ECLA1_v1.prot.fa'
ogfile = '../../../../figshare/orthofinder/Orthogroups/Orthogroups.tsv'
annfile = '../../../../figshare/functional_enrichment_analysis/Elysia_eggnog_annotations.txt'
degfile = '../../../../figshare/gene_expression/DEG_results.txt'
outfile = '../../../../figshare/functional_enrichment_analysis/functional_enrichment_results_KEGG.txt'
obofile = '../../../../figshare/functional_enrichment_analysis/go.obo'

In [3]:
# store set of final genes
geneSet = set()
for record in SeqIO.parse(finfafile, "fasta"):
    name = record.id
    gene = name.split('.')[0]
    #print(gene)
    
    geneSet.add(gene)

In [4]:
goDict = {}

########################################
### Load relationship hash from file ###
########################################

def loadrelationships(file):

    with open(file, "r") as f:
        content = f.read()
        # remove end line characters
        entries = content.split('\n\n')
        for entry in entries:
            col = entry.split('\n')
            
            currentID = ''
            currentParents = set()
            name = ''
            space = ''
            
            for line in col:
                #print(line[0:6])
                if line[0:7] == 'id: GO:':
                    currentID = 'GO:' + line.split(':')[-1]
                    #print(currentID)
                    
                if line[0:6] == 'name: ':
                    name = line.split(': ')[-1]
                    #print(name)

                if line[0:11] == 'namespace: ':
                    space = line.split(': ')[-1]
                    #print(space)

                if line[0:6] == 'is_a: ':
                    parentID = line.split(' ')[1]
                    currentParents.add(parentID)
                    #print(parentID)
        
            goDict[currentID] = [space, name]

loadrelationships(obofile)

In [5]:
goDict['GO:0008061']

['molecular_function', 'chitin binding']

In [6]:
# Read in DEG results

degSet = set()

fi = open(degfile)

for line in fi:
    col = line.rstrip().split('\t')
    if col[0] == 'logFC':
        continue
        
    test = col[4]
    gene = col[5]
    
    if gene not in geneSet:
        print('fuck', gene)
    
    if test == '1':
        #print(gene)
        degSet.add(gene)

fi.close()

In [7]:
# Number of over-represented DEGs
len(degSet)

3318

In [8]:
# Read orthogroup file

# Initial mega set dictionary 
# set0 = list of orthogroups for checking counts
# set1 = list of Eclarki genes in those orthogroups
ogSets = {}
ogSets['gastropods'] = [set(),set(),set()]
ogSets['sacoglossans'] = [set(),set(),set()]
ogSets['alldegs'] = [set(),degSet,set()]
# ogSets['elysids'] = [set(),set()]
# ogSets['longterms'] = [set(),set()]

fi = open(ogfile)

for line in fi:
    col = line.rstrip().split('\t')
    
    if col[0] == 'Orthogroup':
        continue
        
    og = col.pop(0)
    if len(col) < 3:
        continue 
        
    genes = col.pop(2)
    if genes == '':
        continue
    genelist = []
    for gene in genes.split(', '):
        gene = gene.split('.')[0]
        genelist.append(gene)

        if gene not in geneSet:
            print('fuck', gene)
        
    
    # OGs with Acalifornia
    if col[0] != '':
        count = 0
        for sp in col:
            if sp != '':
                count +=1
        if count == 4:
            #print(genelist, col)
            ogSets['gastropods'][0].add(og)
            for gene in genelist:
                ogSets['gastropods'][2].add(gene)
                if gene in degSet:
                    ogSets['gastropods'][1].add(gene)
    else:
        count = 0
        for sp in col:
            if sp != '':
                count +=1
                
        if count == 3:
            ogSets['sacoglossans'][0].add(og)
            for gene in genelist:
                ogSets['sacoglossans'][2].add(gene)
                if gene in degSet:
                    ogSets['sacoglossans'][1].add(gene)

        
# Uncomment if wanting to split the saccoglossid gene families into smaller subsets
#     elif len(col) == 4:
#         #print(genelist, col)
#         ogSets['saccoglossids'][0].add(og)
#         for gene in genelist:
#             ogSets['saccoglossids'][1].add(gene)

#     elif len(col) == 3:
#         #print(genelist, col)
#         ogSets['elysids'][0].add(og)
#         for gene in genelist:
#             ogSets['elysids'][1].add(gene)

#     elif col[1] != '':
#         #print(genelist, col)
#         ogSets['longterms'][0].add(og)
#         for gene in genelist:
#             ogSets['longterms'][1].add(gene)

fi.close()

In [9]:
len(ogSets['sacoglossans'][0])

2165

In [10]:
len(ogSets['sacoglossans'][1])

554

In [11]:
len(ogSets['alldegs'][1])

3318

In [12]:
len(ogSets['sacoglossans'][2])

4208

In [13]:
codeDict = {}
codeDict['gastropods'] = {}
codeDict['sacoglossans'] = {}
codeDict['alldegs'] = {}
# codeDict['elysids'] = {}
# codeDict['longterms'] = {}
codeDict['total'] = {}

lociDict = {}
lociDict['gastropods'] = set()
lociDict['sacoglossans'] = set()
lociDict['alldegs'] = set()
# lociDict['elysids'] = set()
# lociDict['longterms'] = set()
lociDict['total'] = set()

atypeDict = {}

In [29]:
def add_annotation(gene, atype, alist):
    if alist != '-':
        alist = alist.strip('"')
        
        for annotation in alist.split(','):
            if annotation == 'ko:K01379':
                print(annotation)
            if atype == 'KEGG_Pathway':
                if annotation[0] == 'k':
                    continue
            #print(gene, atype, annotation)
            atypeDict[annotation] = atype
            
            if annotation not in codeDict['total']:
                codeDict['total'][annotation] = set()

            codeDict['total'][annotation].add(gene)
            lociDict['total'].add(gene)
            
            for ogset in ogSets:
                #print(ogset)
                if gene in ogSets[ogset][1]:
                    #print(gene, ogset)
                    if annotation not in codeDict[ogset]:
                        codeDict[ogset][annotation] = set()

                    codeDict[ogset][annotation].add(gene)
                    lociDict[ogset].add(gene)


In [30]:
# Get list of all annotations to check for enrichment
fi = open(annfile)

for line in fi:
    if line[0] == '#':
        continue
    
    col = line.rstrip().split('\t')
    
    gene = col[0].split('.')[0]
    
    if gene not in geneSet:
        #print('fuck', gene)
        continue
       
    #add_annotation(gene, 'Gene_Ontology', col[9])
    add_annotation(gene, 'KEGG_ko', col[11])
    #add_annotation(gene, 'KEGG_Pathway', col[12])
    #add_annotation(gene, 'KEGG_Module', col[13])
    add_annotation(gene, 'KEGG_BRITE', col[16])
    #add_annotation(gene, 'CAZy', col[18])
    #add_annotation(gene, 'PFAM', col[20])

fi.close()

ko:K01379
ko:K01379
ko:K01379
ko:K01379
ko:K01379
ko:K01379


In [31]:
codeDict['total']['ko:K01379']

{'Ecla12097g92950',
 'Ecla12097g92960',
 'Ecla2076g320520',
 'Ecla3558g421100',
 'Ecla6219g560940',
 'Ecla6220g560990'}

In [16]:
# Num genes with one or more annotations in set
len(lociDict['sacoglossans'])

70

In [17]:
len(lociDict['gastropods'])

386

In [18]:
len(lociDict['alldegs'])

739

In [19]:
# Number of unique annotations assigned to one or more genes in set
len(codeDict['sacoglossans'])

124

In [20]:
len(codeDict['gastropods'])

504

In [21]:
len(codeDict['alldegs'])

702

In [33]:
df = pd.DataFrame(columns=['set','annotation','category','space','desc','x','N','n','M','pval','genelist'])
highCount = 10000000
lowCount = 0

for ogset in ogSets:
    for annotation in codeDict[ogset]:
        #if annotation == 'ko:K01379':
        #    print(ogset,annotation)

        count = len(codeDict[ogset][annotation])
        if count > highCount or count < lowCount:
            continue

        if annotation == 'ko:K01379':
            print(ogset,annotation)

        cat = atypeDict[annotation]
        #desc = ''
        #if annotation in annotationDict:
        #    desc = annotationDict[annotation]

        # x is still the number of drawn "successes" (ie no. genes in set and in go category)
        x = len(codeDict[ogset][annotation])
        genelist = ', '.join(codeDict[ogset][annotation])

        # N is the sample size (ie no. genes in set)
        N = len(lociDict[ogset])

        # n is the number of successes in the population (ie no. genes in go category)
        n = len(codeDict['total'][annotation])

        # M is the population size (ie no. genes total)
        M = len(lociDict['total'])

        # https://alexlenail.medium.com/understanding-and-implementing-the-hypergeometric-test-in-python-a7db688a7458
        # https://github.com/jdrudolph/goenrich
        pval = hypergeom.sf(x-1, M, n, N)
        if annotation in goDict:
            df.loc[len(df.index)] = [ogset,annotation,cat,goDict[annotation][0],goDict[annotation][1],x,N,n,M,pval,genelist]  
        else:
            df.loc[len(df.index)] = [ogset,annotation,cat,'','',x,N,n,M,pval,genelist]  


gastropods ko:K01379
alldegs ko:K01379


In [25]:
# Adjust pvalues for multiple tests
if len(df['pval'].tolist()) > 0:
    adjpval = multi.multipletests(df['pval'].tolist(), alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)[1]
    df['adjpval'] = adjpval
    df['seqfreq'] = df['x'] / df['N']
    df['totalfreq'] = df['n'] / df['M']

    df = df[['set','annotation','category','space','desc','x','N','seqfreq','n','M','totalfreq','pval','adjpval','genelist']]

    df.to_csv(outfile, sep='\t', index=False)



In [26]:
ogSets

{'gastropods': [{'OG0008268',
   'OG0014088',
   'OG0013533',
   'OG0011649',
   'OG0014752',
   'OG0008065',
   'OG0014773',
   'OG0006787',
   'OG0013553',
   'OG0002073',
   'OG0005475',
   'OG0010115',
   'OG0009422',
   'OG0004224',
   'OG0007823',
   'OG0002689',
   'OG0012423',
   'OG0010568',
   'OG0002675',
   'OG0004303',
   'OG0005208',
   'OG0003811',
   'OG0006784',
   'OG0005300',
   'OG0008853',
   'OG0004471',
   'OG0004000',
   'OG0007880',
   'OG0005367',
   'OG0010688',
   'OG0005789',
   'OG0006561',
   'OG0004444',
   'OG0010448',
   'OG0001567',
   'OG0011086',
   'OG0012940',
   'OG0010591',
   'OG0010823',
   'OG0005680',
   'OG0013765',
   'OG0014632',
   'OG0010500',
   'OG0012406',
   'OG0009323',
   'OG0009527',
   'OG0011154',
   'OG0008285',
   'OG0014254',
   'OG0004142',
   'OG0011597',
   'OG0001757',
   'OG0009315',
   'OG0002552',
   'OG0009378',
   'OG0002080',
   'OG0005335',
   'OG0005128',
   'OG0011456',
   'OG0000194',
   'OG0003048',
   'OG0006